In [2]:
import polars as pl
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
import seaborn.objects as so

/home/ubuntu/thesis-da/lib/python3.10/site-packages/polars/_cpu_check.py:239: RuntimeWarning: Missing required CPU features.

The following required CPU features were not detected:
    ssse3, sse4.1, sse4.2, popcnt
Continuing to use this version of Polars on this processor will likely result in a crash.
Install the `polars-lts-cpu` package instead of `polars` to run Polars with better compatibility.

Hint: If you are on an Apple ARM machine (e.g. M1) this is likely due to running Python under Rosetta.
It is recommended to install a native version of Python that does not run under Rosetta x86-64 emulation.

If you believe this warning to be a false positive, you can set the `POLARS_SKIP_CPU_CHECK` environment variable to bypass this check.

  warnings.warn(
/tmp/ipykernel_6317/3584767584.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and 

In [ ]:
dlp = pl.read_json("")

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

In [3]:
dlp = pl.read_ndjson("../data/raw/flaws_cloudtrail00.ndjson")

In [8]:
dlp.select(pl.col("errorCode").value_counts()).unnest("errorCode")

errorCode,count
str,u32
"""NoSuchCloudFro…",1
"""Client.Invalid…",3
"""AccessDenied""",7268
"""NoSuchDistribu…",1
"""InsufficientDe…",2
"""Client.Invalid…",5
"""Client.Invalid…",6
"""Client.Invalid…",56
"""ReplicationCon…",401


In [17]:
cloudtrails_log_schema = {"Results": pl.List(pl.Struct({
    "eventVersion": pl.Utf8,
    "userIdentity": pl.Struct({
        "type": pl.Utf8,
        "principalId": pl.Utf8,
        "arn": pl.Utf8,
        "accountId": pl.Utf8,
        "invokedby": pl.Utf8,
        "accesskeyId": pl.Utf8,
        "userName": pl.Utf8,
    }),
    "sessionContext": pl.Struct({
        "attributes": pl.Struct({
            "mfaAuthenticated": pl.Utf8,
            "creationDate": pl.Utf8,
        }),
        "sessionIssuer": pl.Struct({
            "type": pl.Utf8,
            "principalId": pl.Utf8,
            "arn": pl.Utf8,
            "accountId": pl.Utf8,
            "userName": pl.Utf8,
        }),
    }),
    "eventTime": pl.Utf8,
    "eventSource": pl.Utf8,
    "eventName": pl.Utf8,
    "awsRegion": pl.Utf8,
    "sourceIPAddress": pl.Utf8,
    "userAgent": pl.Utf8,
    "errorCode": pl.Utf8,
    "errorMessage": pl.Utf8,
    "requestParameters": pl.Utf8,
    "responseElements": pl.Utf8,
    "additionalEventData": pl.Utf8,
    "requestId": pl.Utf8,
    "eventId": pl.Utf8,
    "resources": pl.List(pl.Struct({
        "ARN": pl.Utf8,
        "accountId": pl.Utf8,
        "type": pl.Utf8,
    })),
    "eventType": pl.Utf8,
    "apiVersion": pl.Utf8,
    "readonly": pl.Utf8,
    "recipientAccountId": pl.Utf8,
    "serviceEventDetails": pl.Utf8,
    "sharedEventId": pl.Utf8,
    "vpcEndpointId": pl.Utf8,
}))
}

In [18]:
df = pl.read_json("../data/raw/flaws_cloudtrail05.json", schema=cloudtrails_log_schema)

In [23]:
df.select(pl.col("Results")).unnest("Results")

SchemaError: invalid series dtype: expected `Struct`, got `list[struct[24]]`

In [12]:
dlp.select(pl.col("*")).filter(pl.col("errorCode").is_null())

userAgent,eventID,userIdentity,eventType,sourceIPAddress,eventName,eventSource,recipientAccountId,requestParameters,awsRegion,requestID,responseElements,eventVersion,eventTime,errorMessage,errorCode,apiVersion
str,str,struct[6],str,str,str,str,str,struct[44],str,str,struct[9],str,str,str,str,str
"""[S3Console/0.4…","""3038ebd2-c98a-…","{""Root"",""811596193553"",""arn:aws:iam::811596193553:root"",""811596193553"",null,{{""false"",""2017-02-12T19:57:05Z""}}}","""AwsApiCall""","""255.253.125.11…","""ListBuckets""","""s3.amazonaws.c…","""811596193553""","{null,null,null,{null},{{{null,null},null,null}},null,{null},{null},{null},null,{null},{null},null,null,{null},{null,null},{null},{null},null,null,null,{null},null,null,{null},null,null,{null},{null},null,null,null,null,{null},{null,null},{null},{null},{null},{null},{null},null,{null},{null},null}","""us-east-1""","""83A6C73FE87F51…","{null,null,null,null,null,null,null,null,{null,null,null,null,null,null}}","""1.04""","""2017-02-12T19:…",null,null,null
"""console.amazon…","""9facf7ca-cb76-…","{""Root"",""811596193553"",""arn:aws:iam::811596193553:root"",""811596193553"",""ASIA79EXPHZ8SRL55OOE"",{{""false"",""2017-02-12T19:57:05Z""}}}","""AwsApiCall""","""255.253.125.11…","""GetAccountSumm…","""iam.amazonaws.…","""811596193553""","{null,null,null,{null},{{{null,null},null,null}},null,{null},{null},{null},null,{null},{null},null,null,{null},{null,null},{null},{null},null,null,null,{null},null,null,{null},null,null,{null},{null},null,null,null,null,{null},{null,null},{null},{null},{null},{null},{null},null,{null},{null},null}","""us-east-1""","""b110697b2-f15d…","{null,null,null,null,null,null,null,null,{null,null,null,null,null,null}}","""1.02""","""2017-02-12T19:…",null,null,null
"""console.amazon…","""6596d3b4-7c98-…","{""Root"",""811596193553"",""arn:aws:iam::811596193553:root"",""811596193553"",""ASIA79EXPHZ8SRL55OOE"",{{""false"",""2017-02-12T19:57:05Z""}}}","""AwsApiCall""","""255.253.125.11…","""ListAccountAli…","""iam.amazonaws.…","""811596193553""","{null,null,null,{null},{{{null,null},null,null}},null,{null},{null},{null},null,{null},{null},null,null,{null},{null,null},{null},{null},null,null,null,{null},null,null,{null},null,null,{null},{null},null,null,null,null,{null},{null,null},{null},{null},{null},{null},{null},null,{null},{null},null}","""us-east-1""","""b8382b24-f15d-…","{null,null,null,null,null,null,null,null,{null,null,null,null,null,null}}","""1.02""","""2017-02-12T19:…",null,null,null
"""console.amazon…","""9f9d038c-e5a5-…","{""Root"",""811596193553"",""arn:aws:iam::811596193553:root"",""811596193553"",""ASIA79EXPHZ8SRL55OOE"",{{""false"",""2017-02-12T19:57:05Z""}}}","""AwsApiCall""","""255.253.125.11…","""ListMFADevices…","""iam.amazonaws.…","""811596193553""","{null,null,null,{null},{{{null,null},null,null}},null,{null},{null},{null},null,{null},{null},null,null,{null},{null,null},{null},{null},null,null,null,{null},null,null,{null},null,null,{null},{null},null,null,null,null,{null},{null,null},{null},{null},{null},{null},{null},null,{null},{null},null}","""us-east-1""","""b567111c6-f15d…","{null,null,null,null,null,null,null,null,{null,null,null,null,null,null}}","""1.02""","""2017-02-12T19:…",null,null,null
"""console.amazon…","""4babc3a3-77b1-…","{""Root"",""811596193553"",""arn:aws:iam::811596193553:root"",""811596193553"",""ASIA79EXPHZ8SRL55OOE"",{{""false"",""2017-02-12T19:57:05Z""}}}","""AwsApiCall""","""255.253.125.11…","""ListAccessKeys…","""iam.amazonaws.…","""811596193553""","{null,null,null,{null},{{{null,null},null,null}},null,{null},{null},{null},null,{null},{null},null,null,{null},{null,null},{null},{null},null,null,null,{null},null,null,{null},null,null,{null},{null},null,null,null,null,{null},{null,null},{null},{null},{null},{null},{null},null,{null},{null},null}","""us-east-1""","""b83d3435-f15d-…","{null,null,null,null,null,null,null,null,{null,null,null,null,null,null}}","""1.02""","""2017-02-12T19:…",null,null,null
"""console.amazon…",